In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from datetime import datetime
import collections
import csv
import os.path



In [2]:
df_substance2019 = pd.read_csv('../data/kd2019substance.csv', sep=',')
df_kd2018 = pd.read_csv('../data/kd2018.csv', sep=';')
df_kd2019 = pd.read_csv('../data/kd2019.csv', sep=';')
df_kd2020 = pd.read_csv('../data/kd2020.csv', sep=';')
df_kd2021 = pd.read_csv('../data/kd2021.csv', sep=';')
df_substance2019['leto'] = 2019
df_kd2018['leto'] = 2018
df_kd2019 ['leto'] = 2019
df_kd2020 ['leto'] = 2020
df_kd2021 ['leto'] = 2021
kd_all = [df_kd2018, df_kd2019, df_kd2020, df_kd2021, df_substance2019]
"�"

'�'

In [3]:
kd_victims_other = []
for i in range(len(kd_all)):
    kd_all[i] = kd_all[i][kd_all[i]['VrstaOsebe'].notna()]
    kd_victims_other.append((kd_all[i][kd_all[i]['VrstaOsebe'].str.contains('RTEV')], kd_all[i][~kd_all[i]['VrstaOsebe'].str.contains('RTEV')]))

In [ ]:
#TODO:
#-katere so vrste krminalnih dejanj D
#-koliko je vsakega kdja
#-koliko je vsakega kdja po letih D
#-koliko je vsakega kdja po letih po obcinah
#-koliko je vsakega kdja po letih po regijah
#-koliko je skupaj vsakega kdja po obcinah
#-koliko je skupaj vsakega kdja po regijah
#-najpogostejsi prostori za vsako kd
#-cas od izvedbe kd do obtozitve za kd
#-poskusi kaj z leti, starostni razred, denarna skoda

In [38]:
frequencysKD = []
frequencysdf = []
dejanja = set()
for zrtev, notvictim in kd_victims_other:
    df_temp = notvictim[['ZaporednaStevilkaKD','OpisKD', 'leto', 'UpravnaEnotaStoritve']]
    df_temp[['OpisKD1', 'OpisKD2']] = df_temp['OpisKD'].str.split(' - ', 1, expand=True)
    frequencys = []
    for idx,name in enumerate(df_temp['OpisKD2'].value_counts().index.tolist()):
        frequencys.append([name, df_temp['OpisKD2'].value_counts()[idx]])
    frequencysKD.append(np.array(frequencys))
    frequencysdf.append(pd.DataFrame(np.array(frequencys), columns=['OpisKD2', 'frequency']))
    dejanja = set(dejanje for dejanje in df_temp['OpisKD2'].unique()).union(dejanja)

frequencysdf_concat = frequencysdf[0]
for i in range(len(frequencysdf)-1):
   frequencysdf_concat = frequencysdf_concat.append(frequencysdf[i+1], ignore_index=True)
frequencysdf_group = frequencysdf_concat.groupby(['OpisKD2'])['frequency'].sum()
print(frequencysdf_group)

c:\Users\Irinej\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


OpisKD2
                                                                  12
DAJANJE DARIL ZA NEZAKONITO POSREDOVANJE                      426428
DAJANJE PODKUPNINE                                         175319667
DAV�NA ZATAJITEV                                        324427623479
DETOMOR                                                          221
                                                            ...     
ZLORABA ZNAMENJ ZA POMO� IN NEVARNOST                       38334827
�ALJIVA OBDOL�ITEV                                          38273122
�IKANIRANJE NA DELOVNEM MESTU                               61925055
��UVANJE IN JAVNO POVELI�EVANJE TERORISTI�NIH DEJANJ               1
��UVANJE K NASILNI SPREMEMBI USTAVNE UREDITVE                    212
Name: frequency, Length: 215, dtype: object


In [40]:
#kdji po letih po obcinah
temp_dfs = []
for zrtev, notvictim in kd_victims_other:
    df_temp = notvictim[['ZaporednaStevilkaKD', 'MesecStoritve', 'UraStoritve', 'DanVTednu', 'Povratnik',
                        'OpisKD', 'leto', 'UpravnaEnotaStoritve', 'PUStoritveKD', 'UporabljenoSredstvo1',
                        'UporabljenoSredstvo2', 'UporabljenoSredstvo3', 'UporabljenoSredstvo4']]

    df_temp[['OpisKD1', 'OpisKD2']] = df_temp['OpisKD'].str.split(' - ', 1, expand=True)
    temp_dfs.append(df_temp)




c:\Users\Irinej\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
#                                               OpisKD2 frequency
#0                                              TATVINA      5607
#1                    KR�ITEV TEMELJNIH PRAVIC DELAVCEV      4024
#2                                            GOLJUFIJA      3120
#3                                       VELIKA TATVINA      2662
#4    NEUPRAVI�ENA PROIZVODNJA IN PROMET S PREPOVEDA...      2066
#..                                                 ...       ...
#170                             ONESNA�ENJE PITNE VODE         1
#171  PROTIPRAVNA PRILASTITEV STVARI OB PREISKAVI AL...         1
#172                             KR�ITEV ENAKOPRAVNOSTI         1
#173  KR�ITEV PRAVICE DO PRAVNEGA SREDSTVA ALI PETICIJE         1
#174                                                            1


#                                               OpisKD2 frequency
#0                    KR�ITEV TEMELJNIH PRAVIC DELAVCEV      6069
#1                                              TATVINA      5729
#2                                            GOLJUFIJA      3482
#3                                       VELIKA TATVINA      2522
#4                                   PONAREJANJE LISTIN      1752
#..                                                 ...       ...
#167      ��UVANJE K NASILNI SPREMEMBI USTAVNE UREDITVE         1
#168  SRAMOTITEV SLOVENSKEGA NARODA ALI NARODNIH SKU...         1
#169            OGRO�ANJE OSEB POD MEDNARODNIM VARSTVOM         1
#170  IZDELAVA, PRIDOBITEV IN ODTUJITEV PRIPOMO�KOV ...         1
#171                         ONESNA�ENJE �IVIL ALI KRME         1



#                                              OpisKD2 frequency
#0                                             TATVINA      5706
#1                   KR�ITEV TEMELJNIH PRAVIC DELAVCEV      4650
#2                                           GOLJUFIJA      2697
#3                                      VELIKA TATVINA      2490
#4                                             GRO�NJA      2079
#..                                                ...       ...
#162                                        SRAMOTITEV         1
#163                            FINANCIRANJE TERORIZMA         1
#164                      ZLORABA NOTRANJE INFORMACIJE         1
#165  O�ITANJE KAZNIVEGA DEJANJA Z NAMENOM ZANI�EVANJA         1
#166       MALOMARNO OPRAVLJANJE LEKARNI�KE DEJAVNOSTI         1

In [ ]:
abovex = []
x = 10
for i in range(len(frequencysKD)):
    #pd.DataFrame(frequencysKD[i], columns=['OpisKD', 'frequency']['frequency']>x)
    temp_df = pd.DataFrame(frequencysKD[i], columns=['OpisKD2', 'frequency'])
    temp_df['frequency'] = temp_df['frequency'].astype(int)
    abovex.append(temp_df[temp_df['frequency'] > x])

for df in abovex:
    df.plot(x='OpisKD2', y='frequency', kind='bar')